In [ ]:
# importing necessary libraries
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import cv2
import time
from joblib import dump, load


In [ ]:
def load_images(file, images_name, images, labels, crop=True):
    # file: a text file that has a name of an image and a label
    # images_name, images, labels: lists
    with open(file) as f:
        for line in f:
            image_dir = line.strip("\n").split(":")[0]
            if "XY" in line:
                label = 0
                image_dir = image_dir
            elif "XZ" in line:
                label = 1
                image_dir = f"Processed_Images_XZ\{image_dir}"
            elif "YZ" in line:
                label = 2
                image_dir = f"Processed_Images_YZ\{image_dir}"

            # read the image
            image = cv2.imread(f"{image_dir}")
            # resize to 224 x 224
            if crop:
                image = cv2.resize(image, (224, 224))
            # from BGR to gray
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # append image and label to the list
            images_name.append(image_dir)
            images.append(image)
            labels.append(label)


In [ ]:
images_name, images, labels = [], [], []


In [ ]:
# loading in the images
load_images("XY_labels.txt", images_name, images, labels)
load_images("XZ_labels.txt", images_name, images, labels)
load_images("YZ_labels.txt", images_name, images, labels)


In [ ]:
# normalise and transform to np array
def normalise_images(images, labels):
    # Convert to numpy arrays
    images = np.array(images, dtype=np.float32)
    labels = np.array(labels)
    labels = labels.astype(np.int)

    # Normalise the images
    images /= 255.0

    return images, labels


In [ ]:
images_norm, labels = normalise_images(images, labels)


In [ ]:
# 80-20 train test split
x_train, x_test, y_train, y_test = train_test_split(np.array(images_norm), np.array(labels), train_size=0.8,
                                                    test_size=0.2, random_state=42)
# pre-process into 1-D array
x_train = x_train.reshape(-1, 50176)
x_test = x_test.reshape(-1, 50176)



In [ ]:
# creating and training svm model
svm = SVC()  # default parameters: C=1, kernel="rbf"
svm.fit(x_train, y_train)  # SVM training



In [ ]:
def speed_and_acc(model, X_test, y_test):
    # show test accuracy and number of images processed per second
    start_time = time.time()
    test_acc = accuracy_score(y_test, model.predict(X_test))
    delta_time = time.time() - start_time
    num_img = X_test.shape[0]
    print("--- %s images per second ---" % (num_img/delta_time))
    print(f"test acc: {test_acc}")


In [ ]:
speed_and_acc(svm, x_test, y_test)

In [ ]:
dump(svm, 'SVM_plane_identification_FINAL.joblib')
